<a href="https://colab.research.google.com/github/derek-shing/financialnewscraper/blob/master/SeekingAlpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from  bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table
from sqlalchemy import Column, String, Integer
from sqlalchemy.ext.declarative import declarative_base  
from sqlalchemy.orm import sessionmaker
import datetime


In [0]:
yesterday = datetime.datetime.today().date() -datetime.timedelta(days=1)
yesterday =yesterday.strftime("%m-%d-%Y")

In [49]:
yesterday

'07-23-2019'

In [0]:

today = datetime.datetime.now().strftime("%m-%d-%Y")

In [0]:
url = "https://seekingalpha.com/market-news"

In [0]:
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [0]:
date_list2 = [i.split()[0] for i in date_list]

In [0]:
date_list2

In [0]:
news = soup.find(name="ul", attrs={'class':"item-list",'id':'latest-news-list'})

In [0]:
news.fin

In [0]:
root = "https://seekingalpha.com"
id_list = [i['id'] for i in news.find_all(name='li', attrs={'class':'item'})]
heading_list = [i.text for i in news.find_all('a')]
url_list = [root+i['href'] for i in news.find_all('a')]
date_list = [i['data-last-date'] for i in news.find_all(name='li', attrs={'class':'item'})]


In [0]:
id_list2 = [i.split('-')[2] for i in id_list]

In [0]:
id_list2

In [0]:
def getcontext(url):
  soup = BeautifulSoup(requests.get(url).text)
  contexts = soup.find_all(name='p', attrs={"class":"bullets_li"})
  i=0
  while (len(contexts)==0 and i<20 ):
    soup = BeautifulSoup(requests.get(url).text)
    contexts = soup.find_all(name='p', attrs={"class":"bullets_li"})
    i+=1
  context = [i.text for i in contexts]
  return context

In [18]:
url_list[0]

'https://seekingalpha.com/news/3479478-meeting-huawei-ban-white-house'

In [19]:
len(url_list)

60

In [20]:
getcontext(url_list[2])

['Carlos Ghosn has launched a court case in the Netherlands against Japanese carmakers Nissan (OTCPK:NSANY) and Mitsubishi (OTC:MMTOY), seeking €15M in damages from the carmakers, which ousted him as chairman of their alliance last year on charges of embezzlement.',
 '"In the Netherlands, if you want to fire an executive you have to first tell him what he\'s being accused of, and you have to provide him with the evidence for the accusations. Neither of those things has happened," lawyer Laurens de Graaf told Dutch newspaper NRC.',
 'Click to subscribe to real-time analytics on NSANY',
 'Now read: The Options Market Has A Glimmer Of Hope For Tesla Bulls »',
 'Subscribe for full text news in your inbox']

In [0]:
point_list =[getcontext(i) for i in url_list]

In [0]:
point_list

In [0]:
d={"id":id_list2,"date":date_list2,"heading":heading_list,"url":url_list,"summary":point_list}

In [0]:
df = pd.DataFrame(d)

In [67]:
df.head(5)

,id,date,heading,url,summary
0,3480591,2019-07-24,SEB S.A. ADR reports 1H results,https://seekingalpha.com/news/3480591-seb-s-ad...,"[SEB S.A. ADR (OTCPK:SEBYY): 1H EPS of €2.00, ..."
1,3480590,2019-07-24,Daimler AG reports Q2 results,https://seekingalpha.com/news/3480590-daimler-...,[Daimler AG (OTCPK:DMLRY): Q2 GAAP EPS of -€1....
2,3480589,2019-07-24,Akzo Nobel N.V. reports Q2 results,https://seekingalpha.com/news/3480589-akzo-nob...,[Akzo Nobel N.V. (OTCQX:AKZOF): Q2 Non-GAAP EP...
3,3480569,2019-07-24,Liberty Oilfield Services declares $0.05 dividend,https://seekingalpha.com/news/3480569-liberty-...,[Liberty Oilfield Services (NYSE:LBRT) declare...
4,3480587,2019-07-24,Euronet EPS and revenue in-line,https://seekingalpha.com/news/3480587-euronet-...,[Euronet (NASDAQ:EEFT): Q2 Non-GAAP EPS of $1....


In [0]:
POSTGRES_ADDRESS = 'dbtest.c0odp5hguxv4.us-west-2.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY 
POSTGRES_PORT = '5432' 
POSTGRES_USERNAME = "derek" ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'temp1234' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD 
POSTGRES_DBNAME = 'financial' ## CHANGE THIS TO YOUR DATABASE NAME 
# A long string that contains the necessary Postgres login information 
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT, dbname=POSTGRES_DBNAME)) 

In [22]:
db =create_engine(postgres_str)

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [0]:
base.metadata.drop_all(db)

In [0]:

base = declarative_base()

class News2(base): 
    __tablename__ = 'news'
    new_id = Column(Integer, primary_key=True)
    date = Column(String)
    heading = Column(String)
    url = Column(String)
    summary = Column(String)

Session = sessionmaker(db)  
session = Session()

base.metadata.create_all(db)
session.commit()

for index,row in df.iterrows():
  new = News2(new_id= row['id'], date=row['date'], heading=row['heading'], summary=row['summary'],url=row['url'])  
  session.add(new)  
  session.commit()

# Read
#news_in_db = session.query(News)  
#for new in news_in_db:  
#    print(new.heading)

# Update
#doctor_strange.title = "Some2016Film"  
#session.commit()

# Delete
#session.delete(doctor_strange)  
#session.commit()